In [1]:
!pip install langchain
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:0000:01
  Using cached dataclasses_json-0.5.8-py3-none-any.whl (26 kB)
  Using cached numexpr-2.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (381 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cached pydantic-1.10.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached langchainplus_sdk-0.0.10-py3-none-any.whl (21 kB)
  Using cached marshmallow_enum-1.5.1-py2.py3-none-any.whl (4.2 kB)
  Using cached marshmallow-3.19.0-py3-none-any.whl (49 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Attempting uninstall: numexpr
    Found existing installation: numexpr 2.8.3
    Uninstalling numexpr-2.8.3:
      Successfully uninstalled numexpr-2.8.3
  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
  Using cached tiktoken-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)


Import relevant libraries and define the initial parameters

In [5]:
# imports
import pandas as pd
import tiktoken
import openai

from openai.embeddings_utils import get_embedding

# initialization and embedding model parameters
openai.api_key='demo'

embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

# load the top 80 word for each token
csv_path = "./top_80.csv"
df = pd.read_csv(csv_path)
df.head()

,Unnamed: 0,Top_Words,article_id
0,0,pucher vinzidorf roma frauen pfarrhaus fr weih...,FALTER_201512234F1D30CEA9
1,1,vanillekipferln wildtier darf echt abfassen ad...,FALTER_2015122382119EA87A
2,2,vater eltern mutter wechselmodell zuhause dopp...,FALTER_2015122317D031535C
3,3,eishken tel fisch austern fischgeschft estate ...,FALTER_20151223D853291AAA
4,4,silvester uhr kabarettist 2230 maurer special ...,FALTER_201512238A0DEB6E2D


Calculating the entire token number

In [16]:
encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["tokens"] = df["Top_Words"].apply(lambda x: encoding.encode(x))
df["n_tokens"] = df["tokens"].apply(lambda x: len(x))
#df = df[df.n_tokens <= max_tokens].tail(top_n)
numb_tokens = df["n_tokens"].sum()
df.head()
#print(numb_tokens)

,Unnamed: 0,Top_Words,article_id,n_tokens,tokens
0,0,pucher vinzidorf roma frauen pfarrhaus fr weih...,FALTER_201512234F1D30CEA9,170,"[79, 74712, 24848, 89, 29856, 69, 58567, 20425..."
1,1,vanillekipferln wildtier darf echt abfassen ad...,FALTER_2015122382119EA87A,262,"[16023, 4618, 13694, 809, 2312, 8545, 50435, 8..."
2,2,vater eltern mutter wechselmodell zuhause dopp...,FALTER_2015122317D031535C,168,"[85, 977, 658, 4253, 5318, 466, 584, 91930, 26..."
3,3,eishken tel fisch austern fischgeschft estate ...,FALTER_20151223D853291AAA,231,"[68, 819, 2779, 19227, 282, 16438, 100206, 944..."
4,4,silvester uhr kabarettist 2230 maurer special ...,FALTER_201512238A0DEB6E2D,257,"[35904, 83838, 577, 4171, 75095, 548, 5683, 38..."


Cost estimates

In [12]:
# Model	Usage
# Ada v2 ---$0.0001 / 1K tokens
# Ada v1 --- $0.0040 / 1K tokens
cost_est = (numb_tokens/1000) * 0.0001
print(cost_est)

0.1269128


In [2]:
# encode the row text to tokens to be able to track the used number of tokens
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

#----------------


# the the embedding and saving it into a file
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

# add an additional column to the df with the encoding
df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
df.to_csv('output/embedded_1k_reviews.csv', index=False)
    

In [22]:
import pickle

pkl_openai_path = "./openaiembed.pkl"
with open(pkl_openai_path, 'rb') as file: 
    dt_embed_openai = pickle.load(file)
print(dt_embed_openai[0])   


[-0.3578389585018158, 0.04040989652276039, 0.6254292726516724, -0.43261152505874634, -0.3640924096107483, 0.6023291349411011, -0.40385764837265015, -0.04284945875406265, -0.04878164082765579, 0.5205893516540527, -0.29901590943336487, 0.29117757081985474, 0.27299216389656067, 0.006946341600269079, -0.15166744589805603, -0.4588854908943176, 0.6854178309440613, -0.8913744688034058, -0.08899486064910889, -0.6980218887329102, -0.0029271095991134644, 0.7014836072921753, -0.6258471012115479, 0.44991183280944824, -1.1480038166046143, 0.31530722975730896, 0.24840664863586426, -0.43880438804626465, 0.1991639882326126, -0.46463823318481445, -0.23758390545845032, 0.11562174558639526, -0.021658074110746384, 0.8193053007125854, -0.5185678005218506, -0.11082734912633896, 0.33881378173828125, -0.009175421670079231, 0.1515597403049469, 0.4053214490413666, 0.4665148854255676, 0.602564811706543, 0.2771323323249817, -0.401905357837677, 0.2603040337562561, 0.3918280303478241, -0.21207430958747864, 0.703205

Some information about the open ai embedding models:https://platform.openai.com/docs/guides/embeddings/limitations-risks
The pricing can be found here: https://openai.com/pricing